In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
from torch import nn

from gdeep.models import FFNet

from gdeep.visualisation import  persistence_diagrams_of_activations

from torch.utils.tensorboard import SummaryWriter
from gdeep.data import TorchDataLoader


from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces




In [2]:
writer = SummaryWriter()

In [3]:
dl = TorchDataLoader(name="MNIST")
dl_tr, dl_ts = dl.build_dataloader(batch_size=32)

In [4]:
from gdeep.pipeline import Pipeline

model = nn.Sequential(nn.Flatten(), FFNet(0, arch=[28*28, 200, 32, 10]))

In [5]:
from torch.optim import SGD

print(model)
loss_fn = nn.CrossEntropyLoss()

pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer)

# train the model
pipe.train(SGD, 3, lr=0.1)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): FFNet(
    (layer0): Linear(in_features=784, out_features=200, bias=True)
    (layer1): Linear(in_features=200, out_features=32, bias=True)
    (layer2): Linear(in_features=32, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
Test Error: 50  [57600/60000]
 Accuracy: 37.7%,                 Avg loss: 0.065192 

Epoch 2
-------------------------------
Test Error: 86  [57600/60000]
 Accuracy: 51.3%,                 Avg loss: 0.060942 

Epoch 3
-------------------------------
Test Error: 99  [57600/60000]
 Accuracy: 54.2%,                 Avg loss: 0.060042 

Done!


In [6]:
from gdeep.models import ModelExtractor

me = ModelExtractor(model, loss_fn)

x = next(iter(dl_tr))[0]
list_activations = me.get_activations(x)
len(list_activations)


6

In [7]:
from gudhi import SimplexTree as ST
import networkx as nx
import numpy as np


In [30]:

activations = list_activations
arch = [28*28, 200, 32, 10]

def get_activation_graph(activations,arch, index_batch = 1):
    n_layer = len(arch)
    current_node = 0
    activation_graph = ST()
    edge_list = []
    for i in range(n_layer - 1):
        f = lambda x: (x[0] + current_node, x[1] + current_node)
        G = nx.complete_bipartite_graph(arch[i], arch[i+1])
        l = list(G.edges())
        l = map(f,l)
        edge_list.extend(l)
        current_node += arch[i]
    for edge in edge_list:
        activation_graph.insert(list(edge), 0.0)
    activations_flatten = torch.empty(0)
    for layer in range(n_layer):
        activations_flatten = torch.cat((activations_flatten, activations[layer][index_batch]))
    for neuron in range(activations_flatten.size()[0]):
        activation_graph.insert([neuron], float(activations_flatten[neuron]))
    return activation_graph
    
def gudhi_to_giotto(diagrams_gudhi):
    diagrams_giotto = []
    for diagram in diagrams_gudhi:
        diagram_giotto = []
        for dim, bar in diagram:
            diagram_giotto.append([bar[0],bar[1],dim])
        diagrams_giotto.append(diagram_giotto)
    return np.asarray(diagrams_giotto)

def reverse_diagrams(diagrams):
    diagrams_reversed = []
    for diagram in diagrams:
        diagram_reversed = []
        for b,d,q in diagram:
            diagram_reversed.append([d,b,q])
        diagrams_reversed.append(diagram_reversed)       
    return np.asarray(diagrams_reversed)
    
    
    
        

In [9]:
L = get_activation_graph(activations,arch, index_batch = 1)
L.extend_filtration()
pers = L.extended_persistence()

In [11]:
d = gudhi_to_giotto(pers[3])

from gtda.diagrams import PersistenceEntropy as PE
entropy = PE(normalize = True)
E = entropy.fit_transform(reverse_diagrams(d))    

In [12]:
n_epochs = 20
diagrams = []
for i in range(n_epochs):
    me = ModelExtractor(model, loss_fn)
    x = next(iter(dl_tr))[0]
    activations = me.get_activations(x)
    L = get_activation_graph(activations,arch, index_batch = 1)
    L.extend_filtration()
    pers = L.extended_persistence()
    diagrams.append(pers[3])
    pipe.train(SGD, 2, lr=0.1)
    

Epoch 1
-------------------------------
Test Error: 00  [57600/60000]
 Accuracy: 62.0%,                 Avg loss: 0.057609 

Epoch 2
-------------------------------
Test Error: 44  [57600/60000]
 Accuracy: 62.4%,                 Avg loss: 0.057471 

Done!
Epoch 1
-------------------------------
Test Error: 28  [57600/60000]
 Accuracy: 64.3%,                 Avg loss: 0.056905 

Epoch 2
-------------------------------
Test Error: 14  [57600/60000]
 Accuracy: 64.0%,                 Avg loss: 0.056997 

Done!
Epoch 1
-------------------------------
Test Error: 17  [57600/60000]
 Accuracy: 64.8%,                 Avg loss: 0.056733 

Epoch 2
-------------------------------
Test Error: 29  [57600/60000]
 Accuracy: 64.8%,                 Avg loss: 0.056734 

Done!
Epoch 1
-------------------------------
Test Error: 79  [57600/60000]
 Accuracy: 65.1%,                 Avg loss: 0.056639 

Epoch 2
-------------------------------
Test Error: 08  [57600/60000]
 Accuracy: 65.9%,                 Avg

In [13]:
entropy = PE(normalize = True)
diagrams_giotto = gudhi_to_giotto(diagrams[0])
for i in range(1,len(diagrams)):
    diagrams_giotto.append(gudhi_to_giotto(diagrams[1]))

E = entropy.fit_transform(np.asarray(reverse_diagrams(diagrams_giotto)))

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [31]:
diagrams_giotto = gudhi_to_giotto(diagrams)

In [32]:
E = entropy.fit_transform(np.asarray(reverse_diagrams(diagrams_giotto)))

In [37]:
diagrams[0] == diagrams[2]

True